# Lab 1

**Gruppe**: Dominic Künzi, Jaron Matzinger

In [1]:
from collections import defaultdict
from collections import Counter

In [2]:
word_freqs = {
    'low': 5, 'lowest': 2, 'newer': 6, 'wider': 3, 'new': 2
}

## 3.1 Byte-Pair Encoding

### Algorithm

In [3]:
class BytePairEncoding:
    def __init__(self, corpus, num_merges=10):
        """Initialize with a corpus (word frequency dictionary) and the number of merges."""
        self.word_freqs = {word + "_": freq for word, freq in corpus.items()}
        self.num_merges = num_merges
        self.vocab = set("".join(self.word_freqs.keys()))
        self.vocab = list(self.vocab)
        self.splits = {word: [c for c in word] for word in self.word_freqs.keys()}
        self.merges = {}

    def compute_pair_freqs(self):
        """Count occurrences of adjacent symbol pairs in the corpus."""
        pair_freqs = defaultdict(int)
        for word, freq in self.word_freqs.items():
            split = self.splits[word]
            if len(split) == 1:
                continue
            for i in range(len(split) - 1):
                pair = (split[i], split[i + 1])
                pair_freqs[pair] += freq
        return pair_freqs

    def most_freq_pair(self, pair_freqs):
        """Find the most frequent adjacent pair."""
        if not pair_freqs:
            return None, None
        return max(pair_freqs, key=pair_freqs.get)

    def merge_pair(self, a, b):
        """Merge the most frequent pair into a single token in all words."""
        for word in self.word_freqs:
            split = self.splits[word]
            if len(split) == 1:
                continue

            new_split = []
            i = 0
            while i < len(split):
                if i < len(split) - 1 and split[i] == a and split[i + 1] == b:
                    new_split.append(a + b)
                    i += 2
                else:
                    new_split.append(split[i])
                    i += 1

            self.splits[word] = new_split

    def train(self):
        """Perform BPE merges."""
        for _ in range(self.num_merges):
            pair_freqs = self.compute_pair_freqs()
            best_pair = self.most_freq_pair(pair_freqs)

            if best_pair == (None, None):
                break

            a, b = best_pair
            self.merge_pair(a, b)
            self.merges[best_pair] = a + b
            self.vocab.append(a + b)

            print(f'Merged {best_pair} to {a + b}')
            print(f'Updated vocab: {self.vocab}\n')

    def get_vocab(self):
        """Return the final vocabulary."""
        return self.vocab

    def get_merges(self):
        """Return the merge operations performed."""
        return self.merges

    def encode(self, word):
        """Tokenize a word using the trained BPE tokenizer."""
        word = word + "_"
        split = [c for c in word]
        for (a, b), merged in self.merges.items():
            i = 0
            while i < len(split) - 1:
                if split[i] == a and split[i + 1] == b:
                    split = split[:i] + [merged] + split[i + 2:]
                else:
                    i += 1
        return split

In [4]:
bpe = BytePairEncoding(word_freqs, num_merges=8)
print(f'Initial Vocab: {bpe.get_vocab()}\n')

bpe.train()

print(f'Final Vocab: {bpe.get_vocab()}')
print(f'Merged: {bpe.get_merges()}\n')

word = 'older'
print(f'Tokenized {word} to {bpe.encode(word)}')


Initial Vocab: ['l', 'n', 'o', 'r', 'w', 't', 'd', 'i', '_', 'e', 's']

Merged ('e', 'r') to er
Updated vocab: ['l', 'n', 'o', 'r', 'w', 't', 'd', 'i', '_', 'e', 's', 'er']

Merged ('er', '_') to er_
Updated vocab: ['l', 'n', 'o', 'r', 'w', 't', 'd', 'i', '_', 'e', 's', 'er', 'er_']

Merged ('n', 'e') to ne
Updated vocab: ['l', 'n', 'o', 'r', 'w', 't', 'd', 'i', '_', 'e', 's', 'er', 'er_', 'ne']

Merged ('ne', 'w') to new
Updated vocab: ['l', 'n', 'o', 'r', 'w', 't', 'd', 'i', '_', 'e', 's', 'er', 'er_', 'ne', 'new']

Merged ('l', 'o') to lo
Updated vocab: ['l', 'n', 'o', 'r', 'w', 't', 'd', 'i', '_', 'e', 's', 'er', 'er_', 'ne', 'new', 'lo']

Merged ('lo', 'w') to low
Updated vocab: ['l', 'n', 'o', 'r', 'w', 't', 'd', 'i', '_', 'e', 's', 'er', 'er_', 'ne', 'new', 'lo', 'low']

Merged ('new', 'er_') to newer_
Updated vocab: ['l', 'n', 'o', 'r', 'w', 't', 'd', 'i', '_', 'e', 's', 'er', 'er_', 'ne', 'new', 'lo', 'low', 'newer_']

Merged ('low', '_') to low_
Updated vocab: ['l', 'n', 'o',

## 3.2 WordPiece

### a) Algorithm

In [5]:
class WordPieceTokenizer:
    def __init__(self, corpus, target_vocab_size=15):
        """Initialize with a corpus and target vocabulary size."""
        self.word_freqs = corpus
        self.target_vocab_size = target_vocab_size
        self.vocab = ['[UNK]']
        self.vocab_set = set(self.vocab)
        self.splits = {
            word: [c if i == 0 else f'##{c}' for i, c in enumerate(word)]
            for word in self.word_freqs.keys()
        }
        self.initialize_vocab()

    def initialize_vocab(self):
        """Initialize vocabulary with single-character tokens."""
        alphabet = []
        for word in self.word_freqs.keys():
            if word[0] not in alphabet:
                alphabet.append(word[0])
            for letter in word[1:]:
                if f'##{letter}' not in alphabet:
                    alphabet.append(f'##{letter}')

        alphabet.sort()
        self.vocab += alphabet
        self.vocab_set.update(alphabet)

    def compute_pair_scores(self):
        """Compute scores for adjacent symbol pairs in the corpus."""
        letter_freqs = defaultdict(int)
        pair_freqs = defaultdict(int)

        for word, freq in self.word_freqs.items():
            split = self.splits[word]
            if len(split) == 1:
                letter_freqs[split[0]] += freq
                continue
            for i in range(len(split) - 1):
                pair = (split[i], split[i + 1])
                letter_freqs[split[i]] += freq
                pair_freqs[pair] += freq
            letter_freqs[split[-1]] += freq

        scores = {
            pair: freq / (letter_freqs[pair[0]] * letter_freqs[pair[1]])
            for pair, freq in pair_freqs.items()
            if letter_freqs[pair[0]] and letter_freqs[pair[1]]
        }

        return scores

    def merge_pair(self, a, b):
        """Merge the most frequent pair into a single token."""
        for word in self.word_freqs:
            split = self.splits[word]
            if len(split) == 1:
                continue
            i = 0
            while i < len(split) - 1:
                if split[i] == a and split[i + 1] == b:
                    merge = a + b[2:] if b.startswith("##") else a + b
                    split = split[:i] + [merge] + split[i + 2:]
                else:
                    i += 1
            self.splits[word] = split

    def train(self):
        """Train WordPiece by iteratively merging the highest-scoring pairs."""
        while len(self.vocab) < self.target_vocab_size:
            pair_scores = self.compute_pair_scores()

            if not pair_scores:
                break

            best_pair = max(pair_scores, key=pair_scores.get)
            merged_token = (
                best_pair[0] + best_pair[1][2:]
                if best_pair[1].startswith('##') else best_pair[0] + best_pair[1]
            )

            if merged_token not in self.vocab_set:
                self.vocab.append(merged_token)
                self.vocab_set.add(merged_token)

            self.merge_pair(*best_pair)

            print(f'Merged {best_pair} to {merged_token}')
            print(f'Updated vocab: {self.vocab}\n')

    def get_vocab(self):
        """Return the final vocabulary."""
        return self.vocab

    def encode(self, word):
        """Tokenize a word using the trained WordPiece tokenizer."""
        split = [c if i == 0 else f'##{c}' for i, c in enumerate(word)]
        for token in sorted(self.vocab, key=len, reverse=True):
            new_split = []
            i = 0
            while i < len(split):
                found = False
                for j in range(len(split), i, -1):
                    subword = "".join(split[i:j])
                    if subword in self.vocab_set:
                        new_split.append(subword)
                        i = j
                        found = True
                        break
                if not found:
                    new_split.append(split[i])
                    i += 1
            split = new_split
        return split


In [6]:
wp = WordPieceTokenizer(word_freqs, target_vocab_size=15)
print(f'Initial Vocab: {wp.get_vocab()}\n')

wp.train()

print(f'Final Vocab: {wp.get_vocab()}\n')

word = 'older_'
print(f'Tokenized {word} to {wp.encode(word)}')


Initial Vocab: ['[UNK]', '##d', '##e', '##i', '##o', '##r', '##s', '##t', '##w', 'l', 'n', 'w']

Merged ('##s', '##t') to ##st
Updated vocab: ['[UNK]', '##d', '##e', '##i', '##o', '##r', '##s', '##t', '##w', 'l', 'n', 'w', '##st']

Merged ('w', '##i') to wi
Updated vocab: ['[UNK]', '##d', '##e', '##i', '##o', '##r', '##s', '##t', '##w', 'l', 'n', 'w', '##st', 'wi']

Merged ('wi', '##d') to wid
Updated vocab: ['[UNK]', '##d', '##e', '##i', '##o', '##r', '##s', '##t', '##w', 'l', 'n', 'w', '##st', 'wi', 'wid']

Final Vocab: ['[UNK]', '##d', '##e', '##i', '##o', '##r', '##s', '##t', '##w', 'l', 'n', 'w', '##st', 'wi', 'wid']

Tokenized older_ to ['o', '##l', '##d', '##e', '##r', '##_']


### b) Comparison between BPE and WordPiece

For the comparison we used generated text from a Lorem Ipsum Generator we found online.

In [7]:
corpus = """Lorem ipsum dolor sit amet, consetetur sadipscing elitr, sed diam nonumy eirmod tempor invidunt ut labore et dolore magna aliquyam erat, sed diam voluptua. At vero eos et accusam et justo duo dolores et ea rebum. Stet clita kasd gubergren, no sea takimata sanctus est Lorem ipsum dolor sit amet. Lorem ipsum dolor sit amet, consetetur sadipscing elitr, sed diam nonumy eirmod tempor invidunt ut labore et dolore magna aliquyam erat, sed diam voluptua. At vero eos et accusam et justo duo dolores et ea rebum. Stet clita kasd gubergren, no sea takimata sanctus est Lorem ipsum dolor sit amet. Lorem ipsum dolor sit amet, consetetur sadipscing elitr, sed diam nonumy eirmod tempor invidunt ut labore et dolore magna aliquyam erat, sed diam voluptua. At vero eos et accusam et justo duo dolores et ea rebum. Stet clita kasd gubergren, no sea takimata sanctus est Lorem ipsum dolor sit amet.   

Duis autem vel eum iriure dolor in hendrerit in vulputate velit esse molestie consequat, vel illum dolore eu feugiat nulla facilisis at vero eros et accumsan et iusto odio dignissim qui blandit praesent luptatum zzril delenit augue duis dolore te feugait nulla facilisi. Lorem ipsum dolor sit amet, consectetuer adipiscing elit, sed diam nonummy nibh euismod tincidunt ut laoreet dolore magna aliquam erat volutpat.   

Ut wisi enim ad minim veniam, quis nostrud exerci tation ullamcorper suscipit lobortis nisl ut aliquip ex ea commodo consequat. Duis autem vel eum iriure dolor in hendrerit in vulputate velit esse molestie consequat, vel illum dolore eu feugiat nulla facilisis at vero eros et accumsan et iusto odio dignissim qui blandit praesent luptatum zzril delenit augue duis dolore te feugait nulla facilisi.   

Nam liber tempor cum soluta nobis eleifend option congue nihil imperdiet doming id quod mazim placerat facer possim assum. Lorem ipsum dolor sit amet, consectetuer adipiscing elit, sed diam nonummy nibh euismod tincidunt ut laoreet dolore magna aliquam erat volutpat. Ut wisi enim ad minim veniam, quis nostrud exerci tation ullamcorper suscipit lobortis nisl ut aliquip ex ea commodo consequat.   

Duis autem vel eum iriure dolor in hendrerit in vulputate velit esse molestie consequat, vel illum dolore eu feugiat nulla facilisis.   

At vero eos et accusam et justo duo dolores et ea rebum. Stet clita kasd gubergren, no sea takimata sanctus est Lorem ipsum dolor sit amet. Lorem ipsum dolor sit amet, consetetur sadipscing elitr, sed diam nonumy eirmod tempor invidunt ut labore et dolore magna aliquyam erat, sed diam voluptua. At vero eos et accusam et justo duo dolores et ea rebum. Stet clita kasd gubergren, no sea takimata sanctus est Lorem ipsum dolor sit amet. Lorem ipsum dolor sit amet, consetetur sadipscing elitr, At accusam aliquyam diam diam dolore dolores duo eirmod eos erat, et nonumy sed tempor et et invidunt justo labore Stet clita ea et gubergren, kasd magna no rebum. sanctus sea sed takimata ut vero voluptua. est Lorem ipsum dolor sit amet. Lorem ipsum dolor sit amet, consetetur"""

In [8]:
words = corpus.lower().split()
word_freqs = Counter(words)
print(word_freqs)

Counter({'et': 27, 'dolor': 17, 'lorem': 14, 'ipsum': 14, 'sit': 14, 'sed': 12, 'diam': 12, 'dolore': 12, 'ut': 11, 'amet,': 8, 'at': 8, 'vero': 8, 'ea': 8, 'magna': 7, 'consetetur': 6, 'tempor': 6, 'eos': 6, 'accusam': 6, 'justo': 6, 'duo': 6, 'dolores': 6, 'rebum.': 6, 'stet': 6, 'clita': 6, 'kasd': 6, 'gubergren,': 6, 'no': 6, 'sea': 6, 'takimata': 6, 'sanctus': 6, 'est': 6, 'amet.': 6, 'vel': 6, 'in': 6, 'sadipscing': 5, 'elitr,': 5, 'nonumy': 5, 'eirmod': 5, 'invidunt': 5, 'labore': 5, 'aliquyam': 5, 'erat,': 5, 'voluptua.': 5, 'duis': 5, 'nulla': 5, 'autem': 3, 'eum': 3, 'iriure': 3, 'hendrerit': 3, 'vulputate': 3, 'velit': 3, 'esse': 3, 'molestie': 3, 'consequat,': 3, 'illum': 3, 'eu': 3, 'feugiat': 3, 'facilisis': 2, 'eros': 2, 'accumsan': 2, 'iusto': 2, 'odio': 2, 'dignissim': 2, 'qui': 2, 'blandit': 2, 'praesent': 2, 'luptatum': 2, 'zzril': 2, 'delenit': 2, 'augue': 2, 'te': 2, 'feugait': 2, 'facilisi.': 2, 'consectetuer': 2, 'adipiscing': 2, 'elit,': 2, 'nonummy': 2, 'nibh':

In [9]:
bpe = BytePairEncoding(word_freqs, num_merges=75)
print(f'Initial Vocab: {bpe.get_vocab()}\n')

bpe.train()

print(f'Final Vocab: {bpe.get_vocab()}')
print(f'Merged: {bpe.get_merges()}\n')

word = 'older'
print(f'Tokenized {word} to {bpe.encode(word)}')

Initial Vocab: ['l', 'k', 'v', 'p', 'i', '.', 'x', 'm', 'a', 'o', 'j', 'g', ',', '_', 't', 'e', 'q', 'n', 's', 'f', 'r', 'w', 'b', 'u', 'y', 'd', 'z', 'c', 'h']

Merged ('t', '_') to t_
Updated vocab: ['l', 'k', 'v', 'p', 'i', '.', 'x', 'm', 'a', 'o', 'j', 'g', ',', '_', 't', 'e', 'q', 'n', 's', 'f', 'r', 'w', 'b', 'u', 'y', 'd', 'z', 'c', 'h', 't_']

Merged ('m', '_') to m_
Updated vocab: ['l', 'k', 'v', 'p', 'i', '.', 'x', 'm', 'a', 'o', 'j', 'g', ',', '_', 't', 'e', 'q', 'n', 's', 'f', 'r', 'w', 'b', 'u', 'y', 'd', 'z', 'c', 'h', 't_', 'm_']

Merged ('o', 'r') to or
Updated vocab: ['l', 'k', 'v', 'p', 'i', '.', 'x', 'm', 'a', 'o', 'j', 'g', ',', '_', 't', 'e', 'q', 'n', 's', 'f', 'r', 'w', 'b', 'u', 'y', 'd', 'z', 'c', 'h', 't_', 'm_', 'or']

Merged ('l', 'or') to lor
Updated vocab: ['l', 'k', 'v', 'p', 'i', '.', 'x', 'm', 'a', 'o', 'j', 'g', ',', '_', 't', 'e', 'q', 'n', 's', 'f', 'r', 'w', 'b', 'u', 'y', 'd', 'z', 'c', 'h', 't_', 'm_', 'or', 'lor']

Merged ('a', '_') to a_
Updated

In [10]:
wp = WordPieceTokenizer(word_freqs, target_vocab_size=75)
print(f'Initial Vocab: {wp.get_vocab()}\n')

wp.train()

print(f'Final Vocab: {wp.get_vocab()}\n')

word = 'older_'
print(f'Tokenized {word} to {wp.encode(word)}')

Initial Vocab: ['[UNK]', '##,', '##.', '##a', '##b', '##c', '##d', '##e', '##f', '##g', '##h', '##i', '##k', '##l', '##m', '##n', '##o', '##p', '##q', '##r', '##s', '##t', '##u', '##v', '##x', '##y', '##z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'z']

Merged ('z', '##z') to zz
Updated vocab: ['[UNK]', '##,', '##.', '##a', '##b', '##c', '##d', '##e', '##f', '##g', '##h', '##i', '##k', '##l', '##m', '##n', '##o', '##p', '##q', '##r', '##s', '##t', '##u', '##v', '##x', '##y', '##z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'z', 'zz']

Merged ('##b', '##h') to ##bh
Updated vocab: ['[UNK]', '##,', '##.', '##a', '##b', '##c', '##d', '##e', '##f', '##g', '##h', '##i', '##k', '##l', '##m', '##n', '##o', '##p', '##q', '##r', '##s', '##t', '##u', '##v', '##x', '##y', '##z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 

### 3.3 Other Implementations

We found that implementing subword tokenisation from scratch was way harder than we thought it'd be.\
We spent almost three hours trying to do Byte-Pair Encoding on our own.\
But even after that, we hit a wall with merges and frequency calculations.

Thus, we turned to Hugging Face's (HF) implementation.\
With the guidance of the HF tutorial and some extra help from ChatGPT, we finally got a working version of BPE up and running.\

Having had a play around with BPE, we decided to follow the HF tutorial for WordPiece tokenization.\
This helped us understand the main differences between BPE and WordPiece better.

**Tie-Breaking in BPE:**

1. Lexicographic Order (Default):\
If two pairs have the same frequency/score, the pair that appears first in alphabetical order is chosen.\
Pros: Simple and deterministic\
Cons: Biased towards pairs that appear earlier lexicographically, leading to less optimal merges

1. Left-to-Right Order (First Appearance in Text)
If two pairs have the same score, merge the pair that appears first in the dataset.\
Pros: Preserves natural order of text processing\
Cons: Can lead to inconsistent merges across different datasets

1. Longest Subword First:\
If there's a tie, merge the pair that results in the most frequent subword.\
Pros: Captures frequent structures early, leading to more useful subwords\
Cons: Requires additional frequency calculations, adding computational cost

1. Random Tie-Breaking:\
If there is a tie, randomly choose one of the pairs.\
Pros: Introduces variation, useful for ensemble learning\
Cons: Non-deterministic, leading to different tokenizations for the same dataset


### 3.4 HF Implementations

In [11]:
from transformers import AutoTokenizer

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [18]:
tokenizers = {
'Bert': 'google-bert/bert-base-uncased',
'XLNet': 'xlnet/xlnet-base-cased'
}

In [20]:
for name, hf_tokenizer in tokenizers.items():
    tokenizer = AutoTokenizer.from_pretrained(hf_tokenizer)
    tokens = tokenizer.tokenize(corpus[:511])
    print(f'Tokenizer: {name}\n Vocab: {tokens}')

Tokenizer: Bert
 Vocab: ['lore', '##m', 'ip', '##sum', 'do', '##lor', 'sit', 'am', '##et', ',', 'con', '##set', '##et', '##ur', 'sad', '##ip', '##sc', '##ing', 'eli', '##tr', ',', 'se', '##d', 'dia', '##m', 'non', '##um', '##y', 'e', '##ir', '##mo', '##d', 'tempo', '##r', 'in', '##vid', '##unt', 'ut', 'labor', '##e', 'et', 'do', '##lore', 'magna', 'ali', '##qu', '##yam', 'era', '##t', ',', 'se', '##d', 'dia', '##m', 'vol', '##upt', '##ua', '.', 'at', 've', '##ro', 'e', '##os', 'et', 'acc', '##usa', '##m', 'et', 'just', '##o', 'duo', 'dolores', 'et', 'ea', 're', '##bu', '##m', '.', 'ste', '##t', 'clit', '##a', 'ka', '##sd', 'gu', '##berg', '##ren', ',', 'no', 'sea', 'tak', '##ima', '##ta', 'san', '##ctus', 'est', 'lore', '##m', 'ip', '##sum', 'do', '##lor', 'sit', 'am', '##et', '.', 'lore', '##m', 'ip', '##sum', 'do', '##lor', 'sit', 'am', '##et', ',', 'con', '##set', '##et', '##ur', 'sad', '##ip', '##sc', '##ing', 'eli', '##tr', ',', 'se', '##d', 'dia', '##m', 'non', '##um', '##y', 'e'